In [93]:
import pandas as pd
# https://www.kaggle.com/datasets/rutuspatel/walmart-dataset-retail



In [94]:
# df=pd.read_csv("data_sample/(sample2)retaildata.csv")
df=pd.read_csv("data_sample/(sample1)Walmart.csv")

In [95]:
df.head(5)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [96]:
from dotenv import load_dotenv
import os
from ollama import Client
import json
import pandas as pd

load_dotenv()
api_key = os.getenv("OLLAMA_API_KEY")

client = Client(
    host="https://ollama.com",
    headers={"Authorization": f"Bearer {api_key}"}
)

target_fields = [
    "Region", "Week", "Sales", "Holiday",
    "temperature", "fuel_price", "cpi", "unemployment",
    "Promotion_Flag", "Category"
]

prompt = f"""
You are a data preprocessing assistant.

Your task:
Given the dataset columns: {list(df.columns)},
map each target field from this list: {target_fields}
to the most appropriate column name from the dataset.

Mapping rules:
1. Match based on semantic meaning (e.g., "weekly_sales" → "Sales", "holiday_flag" → "Holiday").
2. If no column matches, set its value to null.
3. Special case for region mapping:
- If there is no "region" column, map "store" (or similar) to "Region".
- If both "region" and "store" exist, keep both by mapping:
    "Region": "region"
    "Store": "store"
    (add "Store" as an extra field in output if needed).
4. Return ONLY a valid JSON dictionary, no explanation or commentary.

Example output:
{{
"Region": "region",
"Store": "store",
"Week": "date",
"Sales": "weekly_sales",
"Holiday": "holiday_flag",
"temperature": "temperature",
"fuel_price": "fuel_price",
"cpi": "cpi",
"unemployment": "unemployment",
"Promotion_Flag": null,
"Category": "product_type"
}}
"""

messages = [{"role": "user", "content": prompt}]
response_text = ""

for part in client.chat("gpt-oss:20b", messages=messages, stream=True):
    response_text += part["message"]["content"]

try:
    mapping = json.loads(response_text)
except json.JSONDecodeError:
    print("Warning: model did not return valid JSON.")
    mapping = {}

valid_map = {k: v for k, v in mapping.items() if v and v in df.columns}

df = df.rename(columns={v: k for k, v in valid_map.items()})

if "Week" in df.columns:
    df["Week"] = pd.to_datetime(df["Week"], dayfirst=True, errors="coerce")
    df["Week"] = df["Week"].dt.strftime("%Y-%m-%d")


if "Holiday" in df.columns:
    df["Holiday"] = df["Holiday"].replace(
        {True: 1, False: 0, "true": 1, "false": 0}
    )
    df["Holiday"] = df["Holiday"].astype(int)

final_cols = [col for col in target_fields if col in df.columns]
final_df = df[final_cols]

print("\nFinal standardized columns:", final_df.columns.to_list())


Final standardized columns: ['Region', 'Week', 'Sales', 'Holiday', 'temperature', 'fuel_price', 'cpi', 'unemployment']


In [98]:
final_df

,Region,Week,Sales,Holiday,temperature,fuel_price,cpi,unemployment
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106
...,...,...,...,...,...,...,...,...
6430,45,2012-09-28,713173.95,0,64.88,3.997,192.013558,8.684
6431,45,2012-10-05,733455.07,0,64.89,3.985,192.170412,8.667
6432,45,2012-10-12,734464.36,0,54.47,4.000,192.327265,8.667
6433,45,2012-10-19,718125.53,0,56.47,3.969,192.330854,8.667


In [102]:
df.isna().sum()


Region          0
Week            0
Sales           0
Holiday         0
temperature     0
fuel_price      0
cpi             0
unemployment    0
dtype: int64

In [99]:
# df=input.copy()

# import pandas as pd
# #Week,Region,Category,Sales,Profit,Promotion_Flag

# if "region" or "regions" not in input.columns.to_list():
#     if "store" in input.columns.to_list():
#         df["store"] = df["store"].astype(str)
#         df["Region"]=df["store"]

# df["Week"] = pd.to_datetime(df["date"], format="%d-%m-%Y")

# df["Sales"] = df["weekly_sales"]
# df["Sales"] = df["Sales"].astype(float)

# df["Holiday"]=df["holiday_flag"]
# df["Holiday"] = df["Holiday"].astype(int)

# macro_index=["fuel_price", "cpi", "unemployment"]


# df=df[["Region","Week","Sales","Holiday"]]
    
# df.head(5)

In [ ]:
from datetime import timedelta
df["Week"] = pd.to_datetime(df["Week"], format="%Y-%m-%d")

cutoff_date = df["Week"].max() - pd.DateOffset(years=1)
recent_data = df[df["Week"] >= cutoff_date]

latest_week = df["Week"].max()
previous_week = latest_week - timedelta(days=7)
quarter_start = latest_week - pd.DateOffset(months=3)

two_weeks_data = df[df["Week"].between(previous_week - timedelta(days=7), latest_week)]
quarter_data = df[df["Week"] >= quarter_start]

summary = recent_data.describe().to_string()
quarter_summary = (
   quarter_data.groupby("Region")["Sales"]
   .sum()
   .sort_values(ascending=False)
   .to_string()
)

prompt = f"""
You are acting as an experienced retail data analyst with access to two years of weekly sales data.
Use the data summaries below to generate analytical insights.

Below is the summary of the most recent year’s sales data:
{summary}

Below is the summary of the most recent quarter's sales data:
{quarter_summary}

Below is the summary of the most two week's sales data: 
{two_weeks_data}

Columns:
- Region: store/region ID
- Week: week of observation
- Sales: weekly total revenue
- Holiday: whether week includes holiday (1=yes, 0=no)
- temperature, fuel_price, cpi, unemployment: macro factors

Your task:
1. Identify 3–5 key business insights from this data. 
- At most 5 insights
- Focus on trends, anomalies, or regional differences.
- Example: “Region 3 showed a 10% increase in sales week-over-week.”
- Short-term: Compare recent 2 weeks (e.g., “Region B’s revenue decreased by 12% week-over-week”)
- Medium-term: Highlight best/worst performing regions in the last quarter (e.g., “Region D led the quarter with 15% growth”)
- Long-term: Note any multi-year trends, seasonality, or correlation with macro factors.
- **Important note:** Give *least priority* to unemployment rate analysis. 
   Only include unemployment-related insights if there is a *clear and significant* relationship with major sales fluctuations.
   If unemployment remains relatively stable or shows no correlation with sales, ignore it in the insights.

2. Provide 1–2 actionable business recommendations.
- At most 2 recommendations.
- Based on insights, suggest next steps (e.g., restocking, marketing push).

Output format (must be plain text): 
**Insights:**
1. ...
2. ...
3. ...

**Recommendations:** 
1. ...
2. ...
"""

response = ""
for part in client.chat("gpt-oss:20b", messages=[{"role": "user", "content": prompt}], stream=True):
   response += part["message"]["content"]
   print(part["message"]["content"], end="", flush=True)




**Insights:**

1. **Regional sales hierarchy** – Region 4 dominates the recent quarter with $29.9 M in sales, more than 12 % above the global quarterly average. In stark contrast, Region 45 posted only $1.0 M, roughly 70 % below the industry mean, indicating a substantial performance gap that warrants attention.

2. **Short‑term swing in top performers** – In the last two weeks, Region 44’s weekly revenue jumped 11.7 % from week 12 (≈$324 k) to week 19 (≈$361 k). Region 45, after a 5.3 % dip from week 12 to 19, rebounded with a 6.8 % rise to week 26, ending the three‑week period with the strongest relative growth among the sampled regions.

3. **Stable but slightly cooling sales in key markets** – Regions 1 and 2 showed consistent, modest declines (<6 %) through the three‑week window, pointing to stable demand but a small seasonal dip as the month progressed. This pattern is likely tied to the onset of cooler weather rather than macro‑economic shock.

4. **Weather‑sales association** –